In [144]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [168]:
cd "/content/drive/My Drive/Colab Notebooks/DataWorkshop-Matrix"

/content/drive/My Drive/Colab Notebooks/DataWorkshop-Matrix


In [147]:
df = pd.read_csv('data/mens_shoes.csv', low_memory = False)
df.shape

(18280, 48)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [150]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [151]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [152]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [153]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [155]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()) )


len(keys)

476

In [156]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [157]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_bracelet style', 'feat_fits most screen size', 'feat_rise',
       'feat_manufacturer part number', 'feat_battery type',
       'feat_stylenumber', 'feat_use', 'feat_release', 'feat_interior pockets',
       'feat_boot height'],
      dtype='object', length=526)

In [0]:
keys_stats = {}
for key in keys:
  keys_stats[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [159]:
{k:v for k,v in keys_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_mpn_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


for keys in key:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [161]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[df.brand == df.feat_brand].shape

(8846, 534)

In [162]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
run_model(['brand_cat'], model)

(-57.308540568178955, 4.208544539662661)

In [163]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,...,feat_target audience,feat_frame material:,feat_fabric care,feat_material type,feat_chest size,feat_batteries required?,feat_animal type,feat_fabric,feat_sock style,feat_issued/ not-issued,feat_item spec attr - style (ebay),feat_item width,feat_designer,feat_materials,feat_labor,feat_ean,feat_inseam,feat_reinforced knee,feat_walmart no.,feat_water resistant,feat_watch band material,feat_age,feat_bracelet style,feat_fits most screen size,feat_rise,feat_manufacturer part number,feat_battery type,feat_stylenumber,feat_use,feat_release,feat_interior pockets,feat_boot height,feat_brand_cat,feat_color_cat,feat_gender_cat,feat_mpn_cat,feat_material_cat,feat_sport_cat,feat_style_cat,feat_boot height_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8190-w-navy-7.5,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,-1,-1,-1,-1
1,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8190-w-navy-7.5,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,-1,-1,-1,-1
2,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,-1,-1,-1,-1,-1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,h

In [164]:
feats_cat = [x for x in df.columns if '_cat' in x and x != 'feat_catalog']
feats = ['brand_cat']
feats += feats_cat
feats = list(set(feats))

model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
run_model(feats, model)

(-57.0906298430206, 4.3693612851779005)

In [165]:
print(df.columns)

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_interior pockets', 'feat_boot height', 'feat_brand_cat',
       'feat_color_cat', 'feat_gender_cat', 'feat_mpn_cat',
       'feat_material_cat', 'feat_sport_cat', 'feat_style_cat',
       'feat_boot height_cat'],
      dtype='object', length=534)


In [166]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100)
m.fit(X, y)

perm = PermutationImportance(m, random_state = 1).fit(X, y);
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2603 ± 0.0085,brand_cat
0.0983 ± 0.0080,feat_material_cat
0.0453 ± 0.0032,feat_gender_cat
0.0192 ± 0.0012,feat_brand_cat
0.0080 ± 0.0007,feat_style_cat
0.0069 ± 0.0009,feat_color_cat
0.0007 ± 0.0001,feat_mpn_cat
0.0003 ± 0.0000,feat_sport_cat
0 ± 0.0000,feat_boot height_cat


In [0]:
!git add matrix1/MensShoesPriceModelTrees.ipynb

In [173]:
!git commit -m "Random Forest model for Men's Shoe Prices"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@972d7f0dd56a.(none)')


In [0]:
!git config --global user.email "justynaandrulewicz94@gmail.com"
!git config --global user.name "Justyna Andrulewicz"


In [176]:
!git push

Everything up-to-date
